# Import packages

In [ ]:
# Python >= 3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn >= 0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tfx==0.21.2
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass

# TensorFlow >= 2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import soundfile as sf
import os
import glob
import tqdm
import json
import librosa
from librosa import display
from pathlib import Path
import IPython.display as pd
import matplotlib.pyplot as plt

from scipy.ndimage.filters import maximum_filter1d
from copy import deepcopy
import joblib
import csv
from madmom.io import midi  # to get ground truth
import warnings

# Print 4 random wave files

In [ ]:
fps = glob.glob('/Volumes/Ext_SSD/e-gmd-v1.0.0/**/*.wav', recursive=True)
fps_random = []
np.random.seed()

# setup subplot 
nrows, ncols = 2, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 6))

# plot some audio waveforms
for r in range(nrows):
    for c in range(ncols):
        fp_random = fps[np.random.randint(len(fps))]
        audio, sr = librosa.core.load(fp_random, duration=1,sr=None)
        ax[r][c].plot(audio, c='k')
        # ax[r][c].axis('off')
        ax[r][c].set_title(Path(fp_random).parts[-2:])
        if r == 0:
            ax[r][c].set_xticks([])
        # save random audio filepaths
        fps_random.append(fp_random)

# Define config file for mel spectogram

In [ ]:
# best-practice: write down your preprocessing config in a dictonary
config = {'sr': 16000, 
          'audio_length': 1,  # in seconds
          'mono': True,
          'n_mels': 64,  # number bins / vertical resolution
          'n_fft': 2048,  # samples for fft
          'hop_length': 441, # shift to the right / frames with 10ms width
          'win_length': 2048,
          'window': 'hann',
          'center': True,
          'pad_mode': 'reflect',
          'power': 2.0,
         }

# save number of frames from length in samples divided by fft hop length
config['n_frames'] = int(config['sr']*config['audio_length']/config['hop_length']) + 1
print(f'config["n_frames"]: {config["n_frames"]}')

# save input shape for model
config['input_shape'] = (config['n_frames'], config['n_mels'],  1)
print(f'config["input_shape"]: {config["input_shape"]}')

# save config 
with open('/Volumes/Ext_SSD/e-gmd-v1.0.0/e-gmd.json', 'w+') as fp:
    json.dump(config, fp, sort_keys=True, indent=4)

# pretty print json
print(json.dumps(config, indent=4))

# STFTs / Mel Spectograms of the 4 randomly chosen wav files

In [ ]:
# setup subplot 
nrows, ncols = 4, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 12))

# plot some audio waveforms
for i, fp_random in enumerate(fps_random):
    audio, sr = librosa.core.load(fp_random, duration=1,sr=config['sr'])

    # calculate stft
    stft = librosa.stft(audio, n_fft=config['n_fft'], hop_length=config['hop_length'], win_length=config['win_length'])
    
    # calculate melspec
    melspec = librosa.feature.melspectrogram(audio, n_fft=config['n_fft'], hop_length=config['hop_length'], win_length=config['win_length'], n_mels=config['n_mels'], fmax=int(config['sr']/2))
    melspec = librosa.amplitude_to_db(melspec, ref=np.max)

    # calculate magnitude and scale to dB
    magspec = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

    # plot with librosa
    librosa.display.specshow(magspec, x_axis='time', y_axis='linear', sr=config['sr'], hop_length=config['hop_length'], ax=ax[i][0])
    librosa.display.specshow(melspec, x_axis='time', y_axis='mel', sr=config['sr'], hop_length=config['hop_length'], ax=ax[i][1])
    
    # adjustments
    # ax[i][1].set_yticks([])
    ax[i][1].set_ylabel(Path(fp_random).parts[-2], rotation=270, labelpad=20)
    ax[i][1].yaxis.set_label_position("right")
    
    # settings for all axises but bottom ones
    if not i == len(fps_random) - 1:
        ax[i][0].set_xticks([])
        ax[i][1].set_xticks([])
        ax[i][0].set_xlabel('')
        ax[i][1].set_xlabel('')
    
    # settings for upper axises
    if i == 0:
        ax[i][0].set_title('stft')
        ax[i][1].set_title('mel spectrogram')   

# adjust whitespace in between subplots        
plt.subplots_adjust(hspace=0.1, wspace=0.1)

print('Melspec shape: %s' % (str(melspec.shape)))
print('Stft shape: %s' % (str(stft.shape)))
print(f'Total data points in mel-spectrogram: {melspec.shape[0]*melspec.shape[1]}')
print(f'Total data points in stft-spectrogram: {stft.shape[0]*stft.shape[1]}')
print(f'-> Data Reduction by factor: {(stft.shape[0]*stft.shape[1]) / (melspec.shape[0]*melspec.shape[1])}')
print()

# Ground truth mapping of MIDI notes 

In [ ]:
drum_mapping={
          36: 0, #kick
          38: 1, #snare head
          40: 2, #snare rim
          37: 3, #snare x-stick
          48: 4, #tom1
          50: 5, #tom1_rim
          45: 6, #tom2
          47: 7, #tom2_rim
          43: 8, #tom3_head
          58: 9, #tom3_rim
          46: 10,#hhopen_bow
          26: 11,#hhopen_edge
          42: 12,#hhclosed_bow
          
          22: 13,#hhclosed_edge
          44: 14,#hhpedal
          49: 15,#crash1_bow
          55: 16,#crash1_edge
          57: 17,#crash2_bow
          52: 18,#crash2_edge
          51: 19,#ride_bow          
          59: 20,#ride_edge
          53: 21,#ride_bell
          
          39: 22,#clap
          54: 23,#tambourine
          56: 24,#cowbell
          70: 25,#maracas
          64: 26,#low_conga
          75: 27 #claves
         }

# save config 
with open('/Volumes/Ext_SSD/e-gmd-v1.0.0/e-gmd.json', 'w+') as fp:
    json.dump(drum_mapping, fp, sort_keys=True, indent=4)

# pretty print json
print(json.dumps(drum_mapping, indent=4))


# Getting ground thruth from MIDI file

In [ ]:
def get_y_from_file(midifile, n_frames):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        pattern = midi.MIDIFile(midifile)
        dt = float(config['hop_length']) / float(config['sr'])       
        
        y_onsets = np.zeros((n_frames, len(drum_mapping)), dtype=np.uint8) #28 instruments deswegen 28-1 
        
        i=0
        for note in pattern.notes:
            instrument=int(note[1])  #1st value in array is for instrument type
            label=drum_mapping[instrument]

            note_start = int(np.round(note[0] / dt)) # 0. value in array is the onset
            i=i+1
            
            if note_start < n_frames:
                y_onsets[note_start, label] = 1
            
            else:
                print('Offset > n_frames reached')
                break

        return y_onsets

In [ ]:
# print onset i[0], label i[1] or duration [2] here 
midi_file_path = glob.glob('/Volumes/Ext_SSD/e-gmd-v1.0.0/**/*.midi', recursive=True)
pattern = midi.MIDIFile(midi_file_path[1])

for i in pattern.notes:
    if (i[0] >= config['audio_length']):
        break
    
    print(i[1])
    print(i[0])

# Define a mel filter matrix

In [ ]:
mel_filter = librosa.filters.mel(config['sr'], 
                                 config['n_fft'], 
                                 n_mels=config['n_mels'], 
                                 fmin=0.0, 
                                 fmax=None, 
                                 htk=False, 
                                 norm='slaney', 
                                 dtype=np.float32)

# Preprocessing 

In [ ]:
def load_and_preprocess_data(file_path):
    # path string is saved as byte array in tf.data.dataset -> convert back to str
    if type(file_path) is not str:
        file_path = file_path.numpy()
        file_path = file_path.decode('utf-8')
    
    
    # load audio data 
    y, _ = librosa.core.load(file_path, sr=config['sr'], mono=config['mono'], offset=0.0, duration=None, 
                             dtype=np.float32, res_type='kaiser_best')

    # calculate stft from audio data
    stft = librosa.core.stft(y, n_fft=config['n_fft'], hop_length=config['hop_length'], 
                             win_length=config['win_length'], window=config['window'], 
                             center=config['center'], dtype=np.complex64, pad_mode=config['pad_mode'])

    # filter stft with mel-filter
    mel_spec = mel_filter.dot(np.abs(stft).astype(np.float32) ** config['power'])
    
    # add channel dimension for conv layer  compatibility
    mel_spec = np.expand_dims(mel_spec, axis=-1)
    
    # get ground truth from file_path string
    midi_file_path = file_path.replace("wav", "midi")
    ground_truth = get_y_from_file(midi_file_path, mel_spec.shape[1]) 
    
    return mel_spec, ground_truth


def preprocessing_wrapper(file_path):
    mel_spec, ground_truth = tf.py_function(load_and_preprocess_data, [file_path], [tf.float32, tf.uint8])
    
    mel_spec = tf.transpose(mel_spec, perm=[1,0,2], conjugate=False, name='transpose')
    mel_spec.set_shape([mel_spec.shape[0], config['n_mels'], 1])
    
    ground_truth.set_shape([mel_spec.shape[0], len(drum_mapping)])
    
    return mel_spec, ground_truth

# Process One Test File

In [ ]:
train_files = glob.glob('/Volumes/Ext_SSD/e-gmd-v1.0.0/Train/*.wav', recursive=True)
num_train_files = len(train_files)

mel_spec, ground_truth = preprocessing_wrapper(train_files[0])

# Save Datasets as tf record files

## Train Files

In [ ]:
# Autotune computation
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Paths of train files
train_files = glob.glob('/Volumes/Ext_SSD/e-gmd-v1.0.0/Train/*.wav', recursive=True)

# Train dataset (file paths)
train_dataset = tf.data.Dataset.list_files(train_files)

# Preprocessing
train_dataset = train_dataset.map(preprocessing_wrapper, num_parallel_calls=AUTOTUNE)

# Slicing
train_dataset = train_dataset.map(lambda spec, label: (tf.signal.frame(spec, config['n_frames'], config['n_frames'], axis=0), tf.signal.frame(label, config['n_frames'], config['n_frames'], axis=0)), num_parallel_calls=AUTOTUNE)

train_dataset = train_dataset.unbatch()  # slicing aufloesen, jedes Beispiel wird einzeln hintereinander 

# Save dataset to disk
!rm -rf /Users/thomas/Documents/TU-Berlin/Faecher/Semester3/Audio_DeepLearning/ADT/TF_rec/tf_train
tf.data.experimental.save(dataset=train_dataset,path=f'/Users/thomas/Documents/TU-Berlin/Faecher/Semester3/Audio_DeepLearning/ADT/TF_rec/tf_train', compression='GZIP')

# Show tensor types and shapes in dataset
print(f'train_dataset.element_spec:\n {train_dataset.element_spec}')

## Test Files

In [ ]:
# Paths of test files
test_files = glob.glob('/Volumes/Ext_SSD/e-gmd-v1.0.0/Test/*.wav', recursive=True)

# Test dataset (file paths)
test_dataset = tf.data.Dataset.list_files(test_files)

# Preprocessing 
test_dataset = test_dataset.map(preprocessing_wrapper, num_parallel_calls=AUTOTUNE)

# Slicing
test_dataset = test_dataset.map(lambda spec, label: (tf.signal.frame(spec, config['n_frames'], config['n_frames'], axis=0), tf.signal.frame(label, config['n_frames'], config['n_frames'], axis=0)), num_parallel_calls=AUTOTUNE)

test_dataset = test_dataset.unbatch()  # slicing aufloesen, jedes Beispiel wird einzeln hintereinander 

# Save dataset to disk
!rm -rf /Users/thomas/Documents/TU-Berlin/Faecher/Semester3/Audio_DeepLearning/ADT/TF_rec/tf_test
tf.data.experimental.save(dataset=test_dataset, path=f'/Users/thomas/Documents/TU-Berlin/Faecher/Semester3/Audio_DeepLearning/ADT/TF_rec/tf_test', compression='GZIP')

# Show tensor types and shapes in dataset
print(f'test_dataset.element_spec:\n {test_dataset.element_spec}')

# Load Datasets from Disk

In [ ]:
# Load train dataset
train_dataset = tf.data.experimental.load(f'/Users/thomas/Documents/TU-Berlin/Faecher/Semester3/Audio_DeepLearning/ADT/TF_rec/tf_train', 
                                    (tf.TensorSpec(shape=(config['n_frames'], config['n_mels'], 1), dtype=tf.float32, name=None), 
                                     tf.TensorSpec(shape=(config['n_frames'], len(drum_mapping)), dtype=tf.uint8, name=None)), 
                                    compression='GZIP')
# Keep dataset in memory
train_dataset = train_dataset.cache()

# Shuffle the data
train_dataset = train_dataset.shuffle(buffer_size=num_train_files)

# Batch examples
train_dataset = train_dataset.batch(64)

# Prefetch
train_dataset = train_dataset.prefetch(AUTOTUNE)

# Test Dataset
test_dataset = tf.data.experimental.load(f'/Users/thomas/Documents/TU-Berlin/Faecher/Semester3/Audio_DeepLearning/ADT/TF_rec/tf_test', 
                                    (tf.TensorSpec(shape=(config['n_frames'], config['n_mels'], 1), dtype=tf.float32, name=None), 
                                     tf.TensorSpec(shape=(config['n_frames'], len(drum_mapping)), dtype=tf.uint8, name=None)), 
                                    compression='GZIP')

# Keep dataset in memory
test_dataset = test_dataset.cache()

# Batch examples
test_dataset = test_dataset.batch(64)

# Prefetch
test_dataset = test_dataset.prefetch(AUTOTUNE)

# Show tensor types and shapes in dataset 
print(f'train_dataset.element_spec:\n {train_dataset.element_spec}')
print(f'test_dataset.element_spec:\n {test_dataset.element_spec}')

# Model: Compile & Fit

# CNN

In [ ]:
# Build model
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=config['input_shape']))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 2)))
model.add(tf.keras.layers.GaussianDropout(0.25))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 2)))
model.add(tf.keras.layers.GaussianDropout(0.25))
model.add(tf.keras.layers.Conv2D(28, (3, 3), activation="relu", padding='same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 16)))
model.add(tf.keras.layers.Reshape((37,-1)))
model.add(tf.keras.layers.Dense(28, activation="sigmoid"))  
model.summary()

# List of metrics
metrics = [tf.keras.metrics.BinaryAccuracy(),
           tf.keras.metrics.TrueNegatives(),
           tf.keras.metrics.TruePositives(),
           tf.keras.metrics.FalseNegatives(),
           tf.keras.metrics.FalsePositives(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
          ]

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)

# Fit model
model.fit(train_dataset, epochs=20)

# Test model
model.evaluate(test_dataset)

## CNN + BiLSTM

In [ ]:
# Build model
model_onset = tf.keras.Sequential()
model_onset.add(tf.keras.Input(shape=config['input_shape']))
model_onset.add(tf.keras.layers.Conv2D(16, (3, 3),activation="relu", padding='same', name='2.'))
model_onset.add(tf.keras.layers.BatchNormalization())
model_onset.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu", padding='same'))
model_onset.add(tf.keras.layers.BatchNormalization())
model_onset.add(tf.keras.layers.MaxPool2D(pool_size=(1, 2)))
model_onset.add(tf.keras.layers.GaussianDropout(0.75))
model_onset.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding='same'))
model_onset.add(tf.keras.layers.BatchNormalization())
model_onset.add(tf.keras.layers.MaxPool2D(pool_size=(1, 2)))
model_onset.add(tf.keras.layers.GaussianDropout(0.75))
model_onset.add(tf.keras.layers.Conv2D(1, (1, 1), activation="relu", padding='same'))
model_onset.add(tf.keras.layers.Reshape((37,-1)))
model_onset.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.5, return_sequences=True)))
model_onset.add(tf.keras.layers.Dense(28, activation="sigmoid"))  
model_onset.summary()

# List of metrics
metrics = [tf.keras.metrics.BinaryAccuracy(),
           tf.keras.metrics.TrueNegatives(),
           tf.keras.metrics.TruePositives(),
           tf.keras.metrics.FalseNegatives(),
           tf.keras.metrics.FalsePositives(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
          ]

# Compile model
model_onset.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)

# Fit model
model_onset.fit(train_dataset, epochs=20)

# Test model
model_onset.evaluate(test_dataset)